Load Data and Assign Random Intervention Dates

In [1]:
import pandas as pd
import numpy as np

# Load the DataFrame
df = pd.read_excel(r'C:\Users\csidd\OneDrive\Desktop\Mservice_req\New Microsoft Excel Worksheet.xlsx')

# Generate random intervention dates for each unique employee within January 2024
np.random.seed(42)
unique_employees = df['Employee'].unique()
intervention_dates = pd.date_range(start='2024-01-01', end='2024-01-31', freq='D')
employee_intervention_dates = pd.DataFrame({
    'Employee': unique_employees,
    'Intervention_date': np.random.choice(intervention_dates, size=len(unique_employees), replace=True)
})

# Merge the intervention dates back into the original DataFrame
df = df.merge(employee_intervention_dates, on='Employee', how='left')


Assign Random Intervention Types and Regions

In [2]:
# List of intervention types to randomly assign
intervention_types = ['MSP', 'DLR', 'RMR', 'GTR', 'ITR', 'WTR', 'APR', 'INC']
df['intervention_type'] = np.random.choice(intervention_types, size=len(df), replace=True)

# Define regions from R1 to R9 and assign each employee one region
regions = [f'R{i}' for i in range(1, 10)]
employee_regions = pd.DataFrame({
    'Employee': unique_employees,
    'Region': np.random.choice(regions, size=len(unique_employees), replace=True)
})

# Merge the region assignments back into the original DataFrame
df = df.merge(employee_regions, on='Employee', how='left')

output_path = r'C:\Users\csidd\OneDrive\Desktop\Mservice_req\Columns_added_with_region.xlsx'
df.to_excel(output_path, index=False)

Data Preprocessing and Create Target Variable

In [3]:
# Convert dates to datetime for consistency
df['act_start_date'] = pd.to_datetime(df['act_start_date'])
df['Intervention_date'] = pd.to_datetime(df['Intervention_date'])

# Sort by employee and start date to ensure chronological order
df = df.sort_values(by=['Employee', 'act_start_date'])

# Calculate the change in performance (e.g., 'actual_duration') before and after the intervention
df['Performance_Change'] = df.groupby('Employee')['actual_duration'].diff()

# Create performance categories based on the change in 'actual_duration'
conditions = [
    (df['Performance_Change'] <= -2),  # Decreased significantly
    (df['Performance_Change'] > -2) & (df['Performance_Change'] <= -0.5),  # Decreased slightly
    (df['Performance_Change'].between(-0.5, 0.5)),  # No change
    (df['Performance_Change'] > 0.5) & (df['Performance_Change'] <= 2),  # Increased slightly
    (df['Performance_Change'] > 2)  # Increased significantly
]
df['Performance_Category'] = np.select(conditions, [0, 1, 2, 3, 4])

# Drop rows where 'Performance_Change' is NaN (i.e., the first job for an employee where no comparison is possible)
df = df.dropna(subset=['Performance_Change'])

output_performance_path = r'C:\Users\csidd\OneDrive\Desktop\Mservice_req\Performance_Change_Categories.xlsx'
df.to_excel(output_performance_path, index=False)

print(f"Excel file saved at: {output_performance_path}")

Excel file saved at: C:\Users\csidd\OneDrive\Desktop\Mservice_req\Performance_Change_Categories.xlsx


Prepare Features and Encode Categorical Variables

In [4]:
from sklearn.preprocessing import LabelEncoder

# Label encode 'intervention_type' and 'Region'
label_encoder = LabelEncoder()
df['intervention_type_encoded'] = label_encoder.fit_transform(df['intervention_type'])
df['Region_encoded'] = label_encoder.fit_transform(df['Region'])

# Define features for the model, including 'actual_duration', 'Days_since_start', 'intervention_type_encoded', and 'Region_encoded'
df['Days_since_start'] = (df['act_start_date'] - pd.Timestamp('2024-01-01')).dt.days
X = df[['actual_duration', 'Days_since_start', 'intervention_type_encoded', 'Region_encoded']]
y = df['Performance_Category']


Train the Random Forest Classifier

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Initialize the Random Forest Classifier with optimized hyperparameters for large datasets
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)  # n_jobs=-1 to use all processors

# Train the model on the training data
rf_model.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, n_jobs=-1, random_state=42)

Step 6: Predict and Evaluate the Model

In [6]:
# Predict on the test data
y_pred = rf_model.predict(X_test)

# Evaluate the model with the additional 'Region' feature included
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Create a DataFrame to store the predictions along with actual values and relevant features
output_df = X_test.copy()
output_df['Actual_Performance_Category'] = y_test
output_df['Predicted_Performance_Category'] = y_pred

# Save the output DataFrame to an Excel file
output_predictions_path = r'C:\Users\csidd\OneDrive\Desktop\Mservice_req\Model_Predictions_Output.xlsx'
output_df.to_excel(output_predictions_path, index=False)

print(f"Predictions Excel file saved at: {output_predictions_path}")


Accuracy: 0.8577565758262611
              precision    recall  f1-score   support

           0       0.54      0.00      0.00      8191
           1       0.50      0.00      0.00      8252
           2       0.90      0.97      0.93    119794
           3       0.50      0.81      0.62      7746
           4       0.76      0.95      0.84      7785

    accuracy                           0.86    151768
   macro avg       0.64      0.55      0.48    151768
weighted avg       0.83      0.86      0.81    151768

Predictions Excel file saved at: C:\Users\csidd\OneDrive\Desktop\Mservice_req\Model_Predictions_Output.xlsx
